In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,3192.71,-0.02,-0.00,0.02,0.00,-0.00,0.00,-0.00,-0.00
3,-0.02,823.41,-0.01,0.00,-0.00,0.00,0.00,0.00,0.00
4,-0.00,-0.01,202.82,0.00,-0.00,-0.00,-0.00,-0.00,0.00
5,0.02,0.00,0.00,47.64,0.00,-0.00,0.00,0.00,0.00
6,0.00,-0.00,-0.00,0.00,10.94,-0.00,-0.00,0.00,0.00
7,-0.00,0.00,-0.00,-0.00,-0.00,2.54,0.00,0.00,0.00
8,0.00,0.00,-0.00,0.00,-0.00,0.00,0.62,0.00,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.16,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.04


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,0.00000,-0.00003,0.00001,-0.00002,-0.00001
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00002,0.00007,0.00001,0.00001
4,-0.00000,-0.00001,1.00000,0.00004,-0.00000,-0.00000,-0.00005,-0.00001,0.00002
5,0.00004,0.00001,0.00004,1.00000,0.00001,-0.00002,0.00007,0.00001,0.00001
6,0.00000,-0.00003,-0.00000,0.00001,1.00000,-0.00001,-0.00001,0.00003,0.00001
7,-0.00003,0.00002,-0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00001,0.00001
8,0.00001,0.00007,-0.00005,0.00007,-0.00001,0.00002,1.00000,0.00003,0.00001
9,-0.00002,0.00001,-0.00001,0.00001,0.00003,0.00001,0.00003,1.00000,-0.00001
10,-0.00001,0.00001,0.00002,0.00001,0.00001,0.00001,0.00001,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.04121894648309327

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.13393430e+06 7.41781190e+04 4.40895322e+03 2.36588186e+02
 1.19844179e+01 6.08887709e-01 3.20376935e-02 1.59334028e-03
 4.49331189e-36]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999373,-0.034461,-0.007888,-0.001840,-0.000423,-0.000099,-0.000024,-0.000006,-0.000002
1,0.034123,0.998671,-0.037679,-0.008264,-0.001895,-0.000439,-0.000105,-0.000027,-0.000007
2,0.009075,0.036995,0.998363,-0.041653,-0.008985,-0.002083,-0.000508,-0.000128,-0.000033
3,0.002469,0.009614,0.040791,0.997869,-0.048719,-0.010656,-0.002565,-0.000649,-0.000168
4,0.000679,0.002637,0.010713,0.047504,0.996798,-0.061685,-0.014001,-0.003517,-0.000914
5,0.000197,0.000759,0.003083,0.013157,0.059581,0.994218,-0.085838,-0.020076,-0.005190
6,0.000062,0.000239,0.000981,0.004158,0.018184,0.081223,0.987093,-0.133073,-0.031505
7,0.000022,0.000086,0.000349,0.001483,0.006461,0.027846,0.118605,0.958610,-0.257255
8,0.000012,0.000047,0.000192,0.000819,0.003568,0.015349,0.063316,0.250884,0.965815


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006268676582574795,
 0.0013289687310746867,
 0.0016369693730697854,
 0.0021305485395756163,
 0.003201663376597663,
 0.005782362162737642,
 0.012906865873420714,
 0.04139009701763319,
 0.03418464841214708]